In [4]:
#pip install pypdf sentencepiece tiktoken openai chromadb sentence_transformers protobuf==3.20.* langchain

## 1.1 Import Libraries

In [5]:
## Importing DS modules
import pandas as pd

# Importing various components from the langchain library
from langchain.document_loaders import PyPDFDirectoryLoader  # For loading documents from a directory
from langchain.document_loaders import PyPDFLoader  # For loading individual PDF documents
from langchain.embeddings import OpenAIEmbeddings  # For creating embeddings using the OpenAI model
from langchain.text_splitter import CharacterTextSplitter  # For splitting text into characters
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For recursively splitting text into characters
from langchain.vectorstores import Chroma  # For storing and retrieving vectors
from langchain.llms import OpenAI  # For using the OpenAI language model
from langchain.chains import ConversationalRetrievalChain  # For creating a retrieval chain for conversation
from langchain.memory import ConversationBufferMemory  # For storing conversation data in memory

# Additional external library imports
import tiktoken  # A tokenization library from OpenAI

## Importing other modules
import copy
import os

## 2.0 Vectorstore Setup

In [6]:
## Path for documents to be retrieved
DOCUMENT_PATH = "data/raw_data"

## Loading documents to be retrieved
loader = PyPDFDirectoryLoader(DOCUMENT_PATH)
documents = loader.load()

## Choosing the tokenizer model from OpenAI
#tiktoken.encoding_for_model("text-davinci-003")
#tokenizer = tiktoken.get_encoding('p50k_base')

## Defining function to split the documents into relevant chunks

def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

## Defining criteria for splitting documents in relevant chunks to be retrieved 

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

## Applying text splitter to our document to be retrieved
documents_splitted = text_splitter.split_documents(documents)

## Setting up environment variables for OpenAI
os.environ["OPENAI_API_KEY"] = ''

## Instantiating the embedding model from OpenAI
embeddings = OpenAIEmbeddings()

## Creating our vectorstore based on the documents_splitted and the OpenAI embedding
vs = Chroma.from_documents(documents_splitted, embeddings)

## 3.0 Pipeline Setup

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Data loading
import pandas as pd

# Underlying ML framework
import torch
from torch import nn
import numpy as np

# progress bars
from tqdm.auto import tqdm

# HuggingFace module for transformer
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

# HuggingFace module for evaluation
#import evaluate

import sentencepiece

MODEL_PATH = "/home/user/code/output_7b_2l_3e"

DEVICE = torch.device("cuda:0")
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to(DEVICE)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=4096,
    temperature=0.2,
    top_p=0.95,
    repetition_penalty=1.15,
    device=0
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [19]:
import time

start_time = time.time()
# Your code snippet goes here

chain = load_qa_chain(local_llm, chain_type="stuff")
query = "Wie viele Wochen Ferien habe ich pro Jahr?"
docs = vs.similarity_search(query, k=2)

prompt_template = """Use the following pieces of context to answer the question at the end:

Context in German: "{context}"

Question in German: "{question}"
Answer in German:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain = load_qa_chain(local_llm, chain_type="stuff", prompt=PROMPT)
result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
print(result)

end_time = time.time()
execution_time = end_time - start_time

print("Execution time: {:.2f} seconds".format(execution_time))

{'output_text': '"Die Regelung besagt, dass Lernende bis zum vollendeten 20. Altersjahr fünf Wochen Ferien haben, während ältere Lernende mindestens vier Wochen Ferien erhalten. Der Ferienanspruch für Lernende gilt pro Bildungsjahr und wird im offiziellen Lehr- oder Anlehrvertragsformular der kantonalen Berufsbildungsämter ein'}
Execution time: 65.39 seconds


## 4.0 Model Evaluation

In [49]:
def generate_context(query, vectorstore, topk):
    
    contexts = ""
    
    docs = vs.similarity_search(query, k=topk)

    for i in range(topk):
        contexts += docs[i].page_content
        
    contexts.replace('\n', '')
    
    return contexts

context = generate_context(query, vs, 5)
context

'gesetzlichen Vertretung).\nErkundigen Sie sich beim Berufsbildungsamt bei der zuständigen Person (Ausbildungsberater/in, \nBerufsinspektor/in). In der Regel wird ein Gespräch mit allen Beteiligten vereinbart.\nStellen Sie allenfalls eine letzte, schriftliche Verwarnung oder Zielvereinbarung aus mit klaren \nZielvorgaben und Auflagen unter Androhung der Lehrvertragsauflösung. Senden Sie eine Kopie \ndieser Verwarnung an den Berufsinspektor resp. die Ausbildungsberaterin.\nBesprechen Sie danach den Stand der Zielerreichung periodisch mit der lernenden Person. Erwähnen \nSie, was sich positiv verändert hat und wo sie welche zusätzlichen Veränderungen wünschen.Hinweis: \nFür das konkrete Vorgehen bei einer Lehrvertragsauflösung gibt die Empfehlung Nr. 5 \n– sbbk.ch/Dokumentation/Empfehlungen/Berufliche Grundbildung/Auflösung der Lehrverträge  – der \nSchweizerischen Berufsbildungsämter-Konferenz SBBK detailliert Auskunft:\n•  Grundlagen und Auflösungsgründe (Erläuterungen, Codes)\n•  Aufl

In [60]:
## Retrieving test data
TEST_PATH = "data/test.csv"
TEST_SAVE_PATH = "data"

test = pd.read_csv(TEST_PATH)

## Name of model currently to be evaluated
model_name = "evaluation_vicuna_finetuned_k=3"

## Collecting context & answers based on test data

answers = []
contexts = []
questions = []

#test2 = test[0:2]

for row in test.itertuples():  
    print('new')
    k = 3
    
    chain = load_qa_chain(local_llm, chain_type="stuff")
    
    ## Retrieving question from test
    query = str(row.Frage)
    docs = vs.similarity_search(query, k=k)
    
    prompt_template = """Use the following pieces of context to answer the question at the end:

    Context in German: "{context}"

    Question in German: "{question}"
    Answer in German:
    """

    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    chain = load_qa_chain(local_llm, chain_type="stuff", prompt=PROMPT)
    result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
    answer = result['output_text'].replace('\n', '')
    print(answer)
    
    ## Appending to the list
    questions.append(query)
    answers.append(answer)
    context = generate_context(query, vs, k)
    contexts.append(context)

## Adding answers to list 
evaluation = pd.DataFrame({'context': contexts, 'question': questions, 'answer': answers})
evaluation.to_csv(os.path.join(TEST_SAVE_PATH, f'{model_name}.csv'), index=False)

new


/home/user/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Ja, es gibt Regelungen für die Nacht- und Sonntagsarbeit von Lehrlingen in der Schweiz. Laut Artikel 14 des Arbeitsgesetzes dürfen Lehrlinge aus besonderen Gründen nur noch bis zu sechs Sonntagen pro Kalenderjahr nachts und sonntags arbeiten. Darüber hinaus gibt es auch spezielle Regelungen für die berufliche Grundbildung sowie für bestimmte Branchen wie die Bäckerei, Konfitüre und Confiserie.
new
1. Lernende können sich an ihre Berufsfachschule
new
1. Den Ausbildungsberater/in des Berufsbildungsamts.
new
In diesem Fall ist die Schule als Lehrbetrieb verpflichtet, die Entschuldig
new
1. 9 Stunden pro Tag und 48 Stunden pro Woche.
new
1. Der Lehrvertrag wird zwischen dem Anbieter der schulisch organisierten Grundbildung und dem Anbieter des Praktikums geschlossen.
new
"Die Berufsmaturitätsschule (BMS) ermöglicht es den Schülern, eine dreijährige berufliche Grundbildung mit eidgenössischem Fähigkeitszeugnis (BM 1) zu absolvieren. Dabei handelt es sich um einen Schulverbund, der von versc

## 5.0 Memory Chain

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(local_llm, vs.as_retriever(search_kwargs={"k": 3}), memory=memory)

## Function to retrieve context and answer from model
def query_model(pipeline, query):
    
    ## Retrieving the answer from model based on query
    result = pipeline({"question":query})

    ## Retrieving the answer for query
    answer = result["answer"]

    return answer

## Setting example query
query = "Beantworte die folgende Frage für einen Schweizer Lehrling auf Deutsch: Kann ich in der Lehre unbezahlten Urlaub nehmen?"

## Prompting our model based on example query
result = query_model(qa, query)

## Retrieving the chat history
memory.load_memory_variables({})

### 5.1 Improved QA Memory Chain

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT


question_generator = LLMChain(llm=local_llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(local_llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=vs.as_retriever(search_kwargs={"k": 3}),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

chat_history = []
query = "Kann ich in der Lehre unbezahlten Urlaub nehmen?"
result = chain({"question": query, "chat_history": chat_history})
result['answer']